### Porting to Google Colab
The following cell enables this notebook to run from Google Colab as well as from your local machine IDE.<br>
You can change `root_directory` and/or `this_notebook_google_path` to point to the directory in your Google account, which contains this notebook, together with the `imgs` sub-directory and the rest of the files.<br>

In [1]:
import sys
import os
try:
    from google.colab import drive as google_drive # type: ignore
except:
    # no Google Colab --> fall back to local machine
    google_drive = None

if google_drive is not None:
    google_drive_directory = os.path.join('/','content','gdrive')
    google_drive.mount(google_drive_directory)
    all_projects_path = os.path.join(google_drive_directory, 'Othercomputers','My Laptop', 'projects')
else:
    all_projects_path = os.path.join('d:\\', 'projects')

project_path = os.path.join(all_projects_path,'RUNI','Thesis')
assert os.path.exists(project_path), f'Project path {project_path} not found!'
# enable import python files from this notebook's path
sys.path.append(project_path)
# enable reading images and data files from this notebook's path
os.chdir(project_path)

datasets_path = os.path.join(project_path, 'datasets')
assert os.path.exists(datasets_path), f'Datasets path {datasets_path} not found!'

output_path = os.path.join(project_path, 'output')
os.makedirs(output_path, exist_ok=True)
assert os.path.exists(output_path), f'Output path {output_path} not found!'

print(f'Current working directory: {os.getcwd()}')
print(f'Datasets path: {datasets_path}')
print(f'Output path: {output_path}')

Current working directory: d:\projects\RUNI\Thesis
Datasets path: d:\projects\RUNI\Thesis\datasets
Output path: d:\projects\RUNI\Thesis\output


In [2]:
import numpy as np

In [3]:
from python.hpc import HybridArray

Numba version: 0.60.0
numba.njit is available.
CUDA is available and will be used for GPU operations.
Printing CUDA active device attributes:
    Name:                               NVIDIA GeForce GTX 1650
    Free Memory:                        3367680 [KB]
    Total Memory:                       4193984 [KB]
    Compute capability:                 7.5
    Clock rate:                         1560.00 [MHz]
    Memory clock rate:                  4001.00 [MHz]
    Memory bus width:                   128 bits
    Memory band width (theoretical)     128.03 [GByte/Sec]
    Number of multiprocessors:          16
    Minimal grid size:                  128
    Maximum grid size:                  (2147483647, 65535, 65535)
    Maximum block dimensions:           (1024, 1024, 64)
    Maximum threads per block:          1024
    Warp size:                          32
    Maximum shared memory per block:    49152 [bytes]
    Maximum registers per block:        65536
    Total constant memory:   

# Testing random integers in series

In [4]:
from python.random_integers.random_integers import random_integers_series

data_py = HybridArray()
data_py.realloc(shape=(10,), dtype=np.uint64, use_gpu=False)
random_integers_series(data=data_py, seed=0, use_njit=False)
print(f'{data_py.numpy()=}')


data_njit = HybridArray()
data_njit.realloc(shape=(10,), dtype=np.uint64, use_gpu=False)
random_integers_series(data=data_njit, seed=0, use_njit=True)
print(f'{data_njit.numpy()=}')

data_gpu = HybridArray()
data_gpu.realloc(shape=(10,), dtype=np.uint64, use_gpu=True)
random_integers_series(data=data_gpu, seed=0)
print(f'{data_gpu.numpy()=}')

data_py.numpy()=array([ 9450065442319768509,  3862337743853815200, 14594248465131455994,
         825350814562895143, 13106526387416306199, 17261738891182278679,
        4740057071801089921,  2162672848317412798, 17314823764052754940,
        9272848099843872704], dtype=uint64)
data_njit.numpy()=array([ 9450065442319768509,  3862337743853815200, 14594248465131455994,
         825350814562895143, 13106526387416306199, 17261738891182278679,
        4740057071801089921,  2162672848317412798, 17314823764052754940,
        9272848099843872704], dtype=uint64)


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


data_gpu.numpy()=array([ 9450065442319768509,  3862337743853815200, 14594248465131455994,
         825350814562895143, 13106526387416306199, 17261738891182278679,
        4740057071801089921,  2162672848317412798, 17314823764052754940,
        9272848099843872704], dtype=uint64)


In [5]:
from python.random_integers.random_integers import splitmix64_matrix
seed_matrix = np.arange(25).reshape(5,5).astype(np.uint64)
print(seed_matrix)
out_states = HybridArray()
out_z = HybridArray()
splitmix64_matrix(states=seed_matrix, out_states=out_states, out_z=out_z, use_njit=False, use_gpu=False)
print('Python:\n' + '='*30)
print('out_states:=\n', out_states.numpy())
print('out_z:=\n', out_z.numpy())
out_states.close()
out_z.close()
splitmix64_matrix(states=seed_matrix, out_states=out_states, out_z=out_z, use_njit=True, use_gpu=False)
print('NJIT:\n' + '='*30)
print('out_states:=\n', out_states.numpy())
print('out_z:=\n', out_z.numpy())
out_states.close()
out_z.close()
splitmix64_matrix(states=seed_matrix, out_states=out_states, out_z=out_z, use_njit=False, use_gpu=True)
print('CUDA:\n' + '='*30)
print('out_states:=\n', out_states.numpy())
print('out_z:=\n', out_z.numpy())
out_states.close()
out_z.close()


[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
Python:
out_states:=
 [[11400714819323198485 11400714819323198486 11400714819323198487
  11400714819323198488 11400714819323198489]
 [11400714819323198490 11400714819323198491 11400714819323198492
  11400714819323198493 11400714819323198494]
 [11400714819323198495 11400714819323198496 11400714819323198497
  11400714819323198498 11400714819323198499]
 [11400714819323198500 11400714819323198501 11400714819323198502
  11400714819323198503 11400714819323198504]
 [11400714819323198505 11400714819323198506 11400714819323198507
  11400714819323198508 11400714819323198509]]
out_z:=
 [[16294208416658607535 10451216379200822465 10905525725756348110
   2092789425003139053  7958955049054603978]
 [ 7134611160154358618 13647215125184110592  7191089600892374487
  11409396526365357622 12587370737594032228]
 [  614480483733483466  5833679380957638813 10682531704454680323
  14180207640020093695  768590962137575583

d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [6]:
from python.random_integers.random_integers import random_integers_base_states_matrix
seed_matrix = np.arange(25).reshape(5,5).astype(np.uint64)
print(seed_matrix)
out_s0 = HybridArray()
out_s1 = HybridArray()
random_integers_base_states_matrix(seeds=seed_matrix, out_s0=out_s0, out_s1=out_s1, use_njit=False, use_gpu=False)
print('Python:\n' + '='*30)
print('out_s0:=\n', out_s0.numpy())
print('out_s1:=\n', out_s1.numpy())
out_s0.close()
out_s1.close()
random_integers_base_states_matrix(seeds=seed_matrix, out_s0=out_s0, out_s1=out_s1, use_njit=True, use_gpu=False)
print('NJIT:\n' + '='*30)
print('out_s0:=\n', out_s0.numpy())
print('out_s1:=\n', out_s1.numpy())
out_states.close()
out_s1.close()
random_integers_base_states_matrix(seeds=seed_matrix, out_s0=out_s0, out_s1=out_s1, use_njit=False, use_gpu=True)
print('CUDA:\n' + '='*30)
print('out_s0:=\n', out_s0.numpy())
print('out_s1:=\n', out_s1.numpy())
out_states.close()
out_s1.close()

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
Python:
out_s0:=
 [[16294208416658607535 10451216379200822465 10905525725756348110
   2092789425003139053  7958955049054603978]
 [ 7134611160154358618 13647215125184110592  7191089600892374487
  11409396526365357622 12587370737594032228]
 [  614480483733483466  5833679380957638813 10682531704454680323
  14180207640020093695  7685909621375755838]
 [ 9753551079159975941  6764836397866521095  9260656408219841379
   1234184003990712370 13564971763896621636]
 [ 3900778703475868044   489215147674969543 14415425345905102346
  16778118630780010966 12306297088033431108]]
out_s1:=
 [[ 7960286522194355700 13757245211066428519 13819372491320860226
  12918135221727111561 16462000697783136304]
 [13877614986023876344  8233034982601383833   309689372594955804
  11288449918072354817 13847876567842155106]
 [13546682927695711814  4839782808629744545 17330038448952222807
   6063221543909367921  1315361123233219194]


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


CUDA:
out_s0:=
 [[16294208416658607535 10451216379200822465 10905525725756348110
   2092789425003139053  7958955049054603978]
 [ 7134611160154358618 13647215125184110592  7191089600892374487
  11409396526365357622 12587370737594032228]
 [  614480483733483466  5833679380957638813 10682531704454680323
  14180207640020093695  7685909621375755838]
 [ 9753551079159975941  6764836397866521095  9260656408219841379
   1234184003990712370 13564971763896621636]
 [ 3900778703475868044   489215147674969543 14415425345905102346
  16778118630780010966 12306297088033431108]]
out_s1:=
 [[ 7960286522194355700 13757245211066428519 13819372491320860226
  12918135221727111561 16462000697783136304]
 [13877614986023876344  8233034982601383833   309689372594955804
  11288449918072354817 13847876567842155106]
 [13546682927695711814  4839782808629744545 17330038448952222807
   6063221543909367921  1315361123233219194]
 [14391278412279324496  1466504191324546123  7220676901988789713
  13174271513393086488  2151

In [7]:
from python.random_integers.random_integers import random_integers_matrix
shape = (5,5)
seed = 0
data_py = HybridArray().realloc(shape=shape, dtype=np.float64, use_gpu=False)
num_steps = np.uint32(0xfffffffe)
num_steps = 10
random_integers_matrix(data=data_py, offset_row0=seed, offset_col0=0, num_steps=num_steps, use_njit=False)
print(f'{data_py.numpy()=}')


data_njit = HybridArray().realloc(shape=shape, dtype=np.float64, use_gpu=False)
random_integers_matrix(data=data_njit, offset_row0=seed, offset_col0=0, num_steps=num_steps, use_njit=True)
print(f'{data_njit.numpy()=}')

data_gpu = HybridArray().realloc(shape=shape, dtype=np.float64, use_gpu=True)
random_integers_matrix(data=data_gpu, offset_row0=seed, offset_col0=0, num_steps=num_steps)
print(f'{data_gpu.numpy()=}')

data_py.numpy()=array([[ 9272848099843872704, 10068280237741184354, 17722164261396592110,
         1974680106086731566, 15796988564049774960],
       [15027215228581985497,  2391319046529270345, 12388166578288778577,
        18223240736954576707,  5985059341229764789],
       [11703315848202772137,  4541498137996601283,   393216479281993992,
        10911565346044368059, 16067918475592983704],
       [ 2569322500284416182,  4781258528930330463, 15651829567618425791,
        13154973360509316237, 11380726293389705028],
       [11837983085756654067, 17141966247246281316,  9643306519567434588,
         2250551252088090109,  1071228636348648322]], dtype=uint64)
data_njit.numpy()=array([[ 9272848099843872704, 10068280237741184354, 17722164261396592110,
         1974680106086731566, 15796988564049774960],
       [15027215228581985497,  2391319046529270345, 12388166578288778577,
        18223240736954576707,  5985059341229764789],
       [11703315848202772137,  4541498137996601283,   39321647

d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


data_gpu.numpy()=array([[ 9272848099843872704, 10068280237741184354, 17722164261396592110,
         1974680106086731566, 15796988564049774960],
       [15027215228581985497,  2391319046529270345, 12388166578288778577,
        18223240736954576707,  5985059341229764789],
       [11703315848202772137,  4541498137996601283,   393216479281993992,
        10911565346044368059, 16067918475592983704],
       [ 2569322500284416182,  4781258528930330463, 15651829567618425791,
        13154973360509316237, 11380726293389705028],
       [11837983085756654067, 17141966247246281316,  9643306519567434588,
         2250551252088090109,  1071228636348648322]], dtype=uint64)
